In [1]:
from PIL import Image
import requests
import io
import math
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
import numpy


torch.set_grad_enabled(False);
import itertools
import seaborn as sns

In [2]:
! pip install git+https://github.com/cocodataset/panopticapi.git

import panopticapi
from panopticapi.utils import id2rgb, rgb2id

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-akk3yate
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-akk3yate
  Resolved https://github.com/cocodataset/panopticapi.git to commit 7bb4655548f98f3fedc07bf37e9040a992b054b0


In [3]:
CLASSES = ['person', ]
# CLASSES = ['N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear','zebra', 'giraffe', 'Backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', ‘snowboard', 'sportsball', 'kite', 'baseballbat','baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', ‘chair', 'couch', 'pottedplant', 'bed', 'N/A', 'diningtable', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', ‘toothbrush']

In [2]:
# Enumerate the above classes(Detectron2 model uses different numbering convention so we need to change it)
coco2d2 = {}
count = 0
for i, c in enumerate(CLASSES):
    if c != "N/A":
        coco2d2[i] = count
        count += 1

# Perform standard PyTorch mean-std input image normalization
transform = T.Compose([T.Resize(800), T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
# Load a pre-trained model from torch hub and request the post-processor
model, postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet101_panoptic', pretrained=True, return_postprocessor=True, num_classes=250)

model.eval()
# Retrieve an image from the validation
# set of COCO dataset for testing purpose
url = "http://images.cocodataset.org/val2017/000000281759.jpg"
im = Image.open(requests.get(url, stream=True).raw)
# Mean-std normalize the input testing image(batch-size: 1)
img = transform(im).unsqueeze(0)
out = model(img)
# Compute the probability score for each possible class, excluding the “no-object” class (the last one)
scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]

# Threshold the confidence to only masks with high confidence > 0/85
keep = scores > 0.85

# Plot the masks satisfying the confidence level condition
ncols = 5
fig, axs = plt.subplots(ncols=ncols, nrows=math.ceil(keep.sum().item()/ncols), figsize=(18, 10))
for line in axs:
    for a in line:
        a.axis('off')
for i, mask in enumerate(out["pred_masks"][keep]):
    ax = axs[i//ncols, i%ncols]
    ax.imshow(mask, cmap="cividis")
    ax.axis('off')
fig.tight_layout()

# Merge the individual predictions obtained by running the above lines of code into a unified panoptic segmentation.For that, we use DETR’s postprocessor.
# The post-processor requires as input the target size of predictions(image sizehere)
result = postprocessor(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]

# Visualize the panoptic segmentation’s results
# The segmentation is stored in a special-format png
panoptic_seg = Image.open(io.BytesIO(result['png_string']))
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8).copy()

# Retrieve the instance id corresponding to each mask
panoptic_seg_id = rgb2id(panoptic_seg)


NameError: name 'CLASSES' is not defined

In [3]:

# Color each mask individually and plot the visualization
palette = [255,0,0]
panoptic_seg[:, :, :] = 0
for id in range(panoptic_seg_id.max()+1):
    panoptic_seg[panoptic_seg_id == id] = numpy.asarray(next(palette))*255
plt.figure(figsize=(15, 15))
plt.imshow(panoptic_seg)
plt.axis('off')
plt.show()

NameError: name 'panoptic_seg' is not defined

In [ ]:
# Use Detectron2’s plotting utilities to better visualize the above panoptic segmentation results.
# !pip install detectron2 == 0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
# from google.colab.patches import cv2_imshow
import cv2

# Extract the segments information and the panoptic result from DETR’s prediction
from copy import deepcopy

In [ ]:

segments_info = deepcopy(result["segments_info"])

# Store the panoptic predictions in a special format png
panoptic_seg = Image.open(io.BytesIO(result['png_string']))
final_w, final_h = panoptic_seg.size

# Convert the png into segment id map
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8)
panoptic_seg = torch.from_numpy(rgb2id(panoptic_seg))

# Change Detectron2’s numbering to appropriate class id’s
meta = MetadataCatalog.get("coco_2017_val_panoptic_separated")
for i in range(len(segments_info)):
    c = segments_info[i]["category_id"]
    segments_info[i]["category_id"] = meta.thing_dataset_id_to_contiguous_id[c] if segments_info[i]["isthing"] else meta.stuff_dataset_id_to_contiguous_id[c]

# Visualize the improved prediction results
v = Visualizer(numpy.array(im.copy().resize((final_w, final_h)))[:, :, ::-1], meta, scale=1.0)
v._default_font_size = 20
v = v.draw_panoptic_seg_predictions(panoptic_seg, segments_info, area_threshold=0)
cv2.imshow(v.get_image())
cv2.waitKey(0)